In [1]:
from defisheye import Defisheye
import numpy as np

In [2]:

dtype = 'linear'
format = 'fullframe'
fov = 190
pfov = 140

img = "/home/cat/Downloads/cohort2/11988.jpg"
img_out = f"/home/cat/Downloads/cohort2/11988{dtype}_{format}_{pfov}_{fov}.jpg"

obj = Defisheye(img, dtype=dtype, format=format, fov=fov, pfov=pfov)
obj.convert(img_out)

AttributeError: 'NoneType' object has no attribute 'shape'

In [86]:
import lensfunpy
import cv2 # OpenCV library

cam_maker = 'NIKON CORPORATION'
cam_model = 'NIKON D3S'
lens_maker = 'Nikon'
lens_model = 'Nikkor 28mm f/1.4D AF'

db = lensfunpy.Database()
cam = db.find_cameras(cam_maker, cam_model)[0]
print (cam)
lens = db.find_lenses(cam, lens_maker, lens_model)[0]
print (lens)



Camera(Maker: Nikon Corporation; Model: Nikon D3S; Mount: Nikon F AF; Crop Factor: 1.0; Score: 0)
Lens(Maker: Nikon; Model: Nikon AF Nikkor 28mm f/1.4D; Type: RECTILINEAR; Focal: 28.0-28.0; Aperture: 1.399999976158142-None; Crop factor: 1.0; Score: 95)


In [85]:
# #print (lensfunpy.Database.find_cameras(maker=None, model=None, loose_search=False))

# db = lensfunpy.Database()
# cam = db.find_cameras('flir')
# print (cam)
# lens = db.find_lenses(cam, 'Nikon', 'Nikkor AF 20mm f/2.8D')[0]



In [6]:
import glob
from tqdm import tqdm, trange
import os
import parmap
import lensfunpy
import cv2 # OpenCV library

focal_length = 4.5
aperture = 1.8
distance = 60
root_dir = '/home/cat/Downloads/cohort2/late/jpgs/'
root_dir_fixed = '//home/cat/Downloads/cohort2/late/jpgs_fixed/'
fnames = np.sort(glob.glob(root_dir+'/*.jpg'))

# 
#
def correct(fnames
            ):

    cam_maker = 'NIKON CORPORATION'
    cam_model = 'NIKON D3S'
    lens_maker = 'Nikon'
    lens_model = 'Nikkor 28mm f/1.4D AF'

    db = lensfunpy.Database()
    cam = db.find_cameras(cam_maker, cam_model)[0]
    lens = db.find_lenses(cam, lens_maker, lens_model)[0]

    for image_path in fnames:
        im_undistorted = cv2.imread(image_path)
        for k in range(6):
            if k==0:
                im_undistorted = np.roll(im_undistorted, -40, 0)
                im_undistorted = np.roll(im_undistorted, -40, 1)

            height, width = im_undistorted.shape[0], im_undistorted.shape[1]

            mod = lensfunpy.Modifier(lens, cam.crop_factor, width, height)
            mod.initialize(focal_length, aperture, distance)

            undist_coords = mod.apply_geometry_distortion()
            im_undistorted = cv2.remap(im_undistorted, undist_coords, None, cv2.INTER_LANCZOS4)

        undistorted_image_path = root_dir_fixed + os.path.split(image_path)[1]
        cv2.imwrite(undistorted_image_path, im_undistorted)

    
if True:
    fnames_split = np.array_split(fnames, 6)
    parmap.map(correct, 
               fnames_split,
               pm_processes=6,
              pm_pbar=True)
    

print ("DONE")
        


100%|██████████| 6/6 [01:37<00:00, 16.21s/it]

DONE
